In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time

크롬 드라이버 설치
https://chromedriver.chromium.org/downloads
압축풀어서 해당 path입력 webdriver.Chrome(path)
동일한 경로면 비우면 됨 webdriver.Chrome()

In [6]:
driver = webdriver.Chrome()

In [8]:
driver.get('https://10times.com/seoul-kr')
time.sleep(1)

<h1>젤 하단까지 스크롤

In [14]:
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(1)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

<h1>Beautiful Soup로 읽어오기

In [119]:
html = driver.page_source

soup = BeautifulSoup(html, 'html.parser')

# 3) 필요한 데이터 검색
title = soup.find('title')
print(title.get_text())

Events in Seoul, Upcoming Events, All Popular Events to Attend in Seoul, South Korea


<h1>원하는 내용 위치 찾아보기

In [120]:
paragraph_data = soup.find_all('tr', {'class':'box'})

print(len(paragraph_data))
row0 = paragraph_data[0]
print(row0)

326
<tr class="box"><td class="text-drkr" data-localizer="ignore"><img alt="" class="lazy pull-right" src="https://im.10times.com/1.png?c=web&amp;s=eventlisting&amp;p=eventsnippet&amp;e=900629,898369,944587&amp;k=40df9820fed3226393d245bbba8a3e7c" style="display: block;"/>Fri, 31 Jan - Mon, 03 Feb 2020<small class="comming text-warning">starting tomorrow</small></td><td><h2 data-edition="900629" id="222357"><a href="https://10times.com/mbc-construction-fair" target="_blank">MBC Construction Fair</a><i class="verified-check tt-svg-sm"></i></h2></td><td> <span class="venue text-drkr">SETEC, Seoul</span></td><td></td><td> <span class="label label-blue">Tradeshow</span> <a class="label label-tag" href="https://10times.com/building-construction">Building &amp; Construction</a></td><td class="tb-foot"> <a class="btn btn-sm btn-default a-m x-es-blb" data-param="interest" href="javascript:void(0)"><i class="fa text-muted fa-star"></i> Interested</a> <a class="btn btn-xs flash" href="https://10t

In [121]:
select_td = row0.select('td')
len(select_td)

6

6일때 원하는 데이터가 있음

In [122]:
html_date = row0.select('td')[0]
print(html_date)
print(html_date.get_text())

<td class="text-drkr" data-localizer="ignore"><img alt="" class="lazy pull-right" src="https://im.10times.com/1.png?c=web&amp;s=eventlisting&amp;p=eventsnippet&amp;e=900629,898369,944587&amp;k=40df9820fed3226393d245bbba8a3e7c" style="display: block;"/>Fri, 31 Jan - Mon, 03 Feb 2020<small class="comming text-warning">starting tomorrow</small></td>
Fri, 31 Jan - Mon, 03 Feb 2020starting tomorrow


<h1>regex로 날짜파싱

In [123]:
import re

strDate = html_date.get_text()
listDate = re.findall('(?P<date>\w{3}, \d{2})\s?(?P<month>\w{3})?\s?(?P<year>\d{4})?', strDate)
print(listDate)

[('Fri, 31', 'Jan', ''), ('Mon, 03', 'Feb', '2020')]


<h1>title 링크

In [124]:
html_title = row0.select('td')[1]
print(html_title)
print(html_title.get_text())

link_title = html_title.select('a')[0].get('href')
print(link_title)

<td><h2 data-edition="900629" id="222357"><a href="https://10times.com/mbc-construction-fair" target="_blank">MBC Construction Fair</a><i class="verified-check tt-svg-sm"></i></h2></td>
MBC Construction Fair
https://10times.com/mbc-construction-fair


<h2>place정보

In [125]:
html_place = row0.select('td')[2]
print(html_place)
print(html_place.get_text())

<td> <span class="venue text-drkr">SETEC, Seoul</span></td>
 SETEC, Seoul


<h2>title2 정보

In [126]:
html_title2 = row0.select('td')[3]
print(html_title2)
print(html_title2.get_text())

<td></td>



<h2>타입 & 카테고리 정보

In [127]:
html_category = row0.select('td')[4]
print(html_category)
print(html_category.get_text())

<td> <span class="label label-blue">Tradeshow</span> <a class="label label-tag" href="https://10times.com/building-construction">Building &amp; Construction</a></td>
 Tradeshow Building & Construction


<h3>타입 정보

In [128]:
html_category_span = html_category.select('span')
print(len(html_category_span))
print(html_category_span[0].get_text())

1
Tradeshow


<h3>카테고리 정보

In [129]:
html_category_a = html_category.select('a')
print(len(html_category_a))
print(html_category_a[0].get_text())

1
Building & Construction


<H1> 정리해서 클래스로 

In [130]:
class CEvent_Row:
    listDate = []
    listType = []
    listCategory = []
    def __init__(self, listTd):
        strDate = listTd[0].get_text()
        self.listDate = re.findall('(?P<date>\w{3}, \d{2})\s?(?P<month>\w{3})?\s?(?P<year>\d{4})?', strDate)
        self.strTitle = listTd[1].get_text()
        self.linkTitle = listTd[1].select('a')[0].get('href')
        self.strPlace = listTd[2].get_text()
        self.strTitle2 = listTd[3].get_text()
        listHtmlType = listTd[4].select('span')
        for itHtmlType in listHtmlType:
            strType = itHtmlType.getText()
            self.listType.append(strType)
            
        listHtmlCategory = listTd[4].select('a')
        for itHtmlCategory in listHtmlCategory:
            strCategory = itHtmlCategory.getText()
            self.listCategory.append(strCategory)

In [131]:
iIndex = 0
for itParagraph in paragraph_data:
    listTD = itParagraph.select('td')
    
    # 6이 아닌 경우 중간광고
    if len(listTD) != 6:
        continue
        
    row = CEvent_Row(listTD)
#     print(row.listDate, row.strTitle, row.linkTitle, row.strPlace, row.strTitle2, row.listType, row.listCategory)
    print(row.listDate)
    iIndex+=1

[('Fri, 31', 'Jan', ''), ('Mon, 03', 'Feb', '2020')]
[('Fri, 31', 'Jan', ''), ('Mon, 03', 'Feb', '2020')]
[('Mon, 03', 'Feb', '2020')]
[('Tue, 04', '', ''), ('Wed, 05', 'Feb', '2020')]
[('Tue, 04', '', ''), ('Wed, 05', 'Feb', '2020')]
[('Wed, 05', '', ''), ('Fri, 07', 'Feb', '2020')]
[('Sat, 08', 'Feb', '2020')]
[('Tue, 11', 'Feb', '2020')]
[('Tue, 11', '', ''), ('Wed, 12', 'Feb', '2020')]
[('Tue, 11', '', ''), ('Wed, 12', 'Feb', '2020')]
[('Tue, 11', '', ''), ('Wed, 12', 'Feb', '2020')]
[('Tue, 11', '', ''), ('Wed, 12', 'Feb', '2020')]
[('Tue, 11', '', ''), ('Wed, 12', 'Feb', '2020')]
[('Tue, 11', '', ''), ('Fri, 14', 'Feb', '2020')]
[('Wed, 12', '', ''), ('Fri, 14', 'Feb', '2020')]
[('Wed, 12', '', ''), ('Fri, 14', 'Feb', '2020')]
[('Thu, 13', '', ''), ('Sun, 16', 'Feb', '2020')]
[('Fri, 14', '', ''), ('Sun, 16', 'Feb', '2020')]
[('Thu, 20', 'Feb', '2020')]
[('Thu, 20', 'Feb', '2020')]
[('Thu, 20', '', ''), ('Fri, 21', 'Feb', '2020')]
[('Fri, 21', '', ''), ('Sat, 22', 'Feb', '2020')]

날짜 적혀있는 패턴이 다양

1. 하루인 경우 : 그날만 써있음
2. 시작하는 날과 끝나는 날의 달이 같은 경우 : 시작날짜에 연도, 월 이 없음
3. 시작하는 날과 끝나는 날의 달이 다르고 년이 같은 경우 : 시작날짜에 연도가 없음

<h1>클래스 수정 (날짜 파싱 함수 추가)

In [132]:
class CEvent_Row:
    listStartEnd = []
    listType = []
    listCategory = []
    def __init__(self, listTd):
        strDate = listTd[0].get_text()
        listDate = re.findall('(?P<date>\w{3}, \d{2})\s?(?P<month>\w{3})?\s?(?P<year>\d{4})?', strDate)
        self.listStartEnd = DateParse(listDate)
        self.strTitle = listTd[1].get_text()
        self.linkTitle = listTd[1].select('a')[0].get('href')
        self.strPlace = listTd[2].get_text()
        self.strTitle2 = listTd[3].get_text()
        listHtmlType = listTd[4].select('span')
        for itHtmlType in listHtmlType:
            strType = itHtmlType.getText()
            self.listType.append(strType)
            
        listHtmlCategory = listTd[4].select('a')
        for itHtmlCategory in listHtmlCategory:
            strCategory = itHtmlCategory.getText()
            self.listCategory.append(strCategory)
    
    def DateParse(listDate): 
        start_day = listDate[0][0]
        start_month = listDate[0][1]
        start_year = listDate[0][2]

        if len(listDate) == 1: #only one day
            strStartDate = start_day + ' ' + start_month + ' ' + start_year
            strEndDate = strStartDate

        elif len(listDate) == 2: #more then two days
            end_day = listDate[1][0]
            end_month = listDate[1][1]
            end_year = listDate[1][2]

            if listDate[0][1] == '':
                start_month = end_month
            if listDate[0][2] == '':
                start_year = end_year

            strStartDate = start_day + ' ' + start_month + ' ' + start_year
            strEndDate = end_day + ' ' + end_month + ' ' + end_year

        listReturn = []
        listReturn.append(strStartDate)
        listReturn.append(strEndDate)
        return listReturn

<h1>DataFrame에 넣기

In [133]:
from pandas import DataFrame

In [134]:
df = DataFrame(columns=('title','title_link', 'start_date', 'end_date','type','category','place','title2'))

In [135]:
iIndex = 0
for itParagraph in paragraph_data:
    listTd = itParagraph.select('td')
    if len(listTd) != 6:
        continue
    
    row = CEvent_Row(listTd)
    
    df.at[iIndex, 'title'] = row.strTitle
    df.at[iIndex, 'title_link'] = row.linkTitle
    df.at[iIndex, 'start_date'] = row.listStartEnd[0]
    df.at[iIndex, 'end_date'] = row.listStartEnd[1]
    df.at[iIndex, 'place'] = row.strPlace
    df.at[iIndex, 'title2'] = row.strTitle2
    df.at[iIndex, 'type'] = row.listType[0]
    df.at[iIndex, 'category'] = row.listCategory
    iIndex += 1

In [136]:
df

,title,title_link,start_date,end_date,type,category,place,title2
0,MBC Construction Fair,https://10times.com/mbc-construction-fair,"Fri, 31 Jan 2020","Mon, 03 Feb 2020",Tradeshow,"[Building & Construction, Furnishings & Decor,...","SETEC, Seoul",
1,Dong a Home & Living Fair,https://10times.com/home-living-fair,"Fri, 31 Jan 2020","Mon, 03 Feb 2020",Tradeshow,"[Building & Construction, Furnishings & Decor,...","iti the center (AT Center), Seoul",Dong a Home & Living Fair will provide correct...
2,International Conference on Business Managemen...,https://10times.com/icbmss-seoul,"Mon, 03 Feb 2020","Mon, 03 Feb 2020",Tradeshow,"[Building & Construction, Furnishings & Decor,...","Hotel Skypark Kingstown Dongdaemun, Seoul",The key motive of ICBMSS is to provide a world...
3,"International Conference on Economics, Managem...",https://10times.com/icemss-seoul,"Tue, 04 Feb 2020","Wed, 05 Feb 2020",Tradeshow,"[Building & Construction, Furnishings & Decor,...",SOTETSU HOTELS THE SPLAISIR SEOUL MYEONG-DONG...,"International Conference on Economics, Managem..."
4,International Conference on Chemical and Bioch...,https://10times.com/icc-be,"Tue, 04 Feb 2020","Wed, 05 Feb 2020",Tradeshow,"[Building & Construction, Furnishings & Decor,...","SOTETSU HOTELS THE SPLAISIR SEOUL DONGDAEMUN,...",International Conference on Chemical and Bioch...
...,...,...,...,...,...,...,...,...
291,Scaffolding Learning through Material Play (So...,https://10times.com/material-play-seoul,"Wed, 09 Dec 2020","Thu, 10 Dec 2020",Tradeshow,"[Building & Construction, Furnishings & Decor,...",Seoul,Professional Development for Teachers
292,ICA Congress,https://10times.com/ica-congress-seoul,"Fri, 11 Dec 2020","Fri, 11 Dec 2020",Tradeshow,"[Building & Construction, Furnishings & Decor,...","COEX Mall, Seoul",ICA Congress works with global and regional go...
293,Reggio Inspired Pedagogy (South Korea)20201214...,https://10times.com/reggio-inspired-pedagogy-s...,"Mon, 14 Dec 2020","Wed, 16 Dec 2020",Tradeshow,"[Building & Construction, Furnishings & Decor,...",Seoul,Professional Development for Teachers
294,Making Learning Visible (South Korea) (MLV)202...,https://10times.com/making-learning-visible-seoul,"Thu, 17 Dec 2020","Fri, 18 Dec 2020",Tradeshow,"[Building & Construction, Furnishings & Decor,...",Seoul,Professional Development for Teachers


In [66]:
df.to_csv("EventInSeoul.csv", sep='\t')